##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorBoard 시작하기

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tensorboard/get_started"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/get_started.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/tensorboard/blob/master/docs/get_started.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/tensorboard/docs/get_started.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a>
  </td>
</table>

머신러닝에서 무언가를 개선하려면 측정할 수 있어야 합니다. TensorBoard는 머신러닝 워크플로 중에 필요한 측정 및 시각화를 제공하는 도구입니다. 손실 및 정확성과 같은 실험 메트릭을 추적하고, 모델 그래프를 시각화하고, 임베딩을 더 낮은 차원의 공간에 투영하는 등의 작업을 수행할 수 있습니다.

이 빠른 시작에서는 TensorBoard를 빠르게 시작하는 방법을 안내합니다. 이 웹 사이트의 나머지 가이드에서는 여기서 다루지 않은 특정 기능에 대한 자세한 정보를 제공합니다. 

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import tensorflow as tf
import datetime

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

[MNIST](https://en.wikipedia.org/wiki/MNIST_database) 데이터세트를 예로 사용하여 데이터를 정규화하고 이미지를 10개의 클래스로 분류하기 위한 간단한 Keras 모델을 생성하는 함수를 작성합니다.

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

11493376/11490434 [==============================] - 0s 0us/step


## Keras Model.fit()과 함께 TensorBoard 사용하기

Keras의 [Model.fit()](https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#fit)으로 훈련하면서 `tf.keras.callbacks.TensorBoard` 콜백을 추가하면 로그를 생성하고 보관할 수 있습니다. 또한, `histogram_freq=1`(기본적으로 해제되어 있음)을 이용해 epoch마다 히스토그램 계산을 사용합니다.

여러 훈련 실행을 쉽게 선택할 수 있도록 로그를 타임스탬프가 있는 하위 디렉토리에 배치합니다.

In [ ]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 15s 246us/sample - loss: 0.2217 - accuracy: 0.9343 - val_loss: 0.1019 - val_accuracy: 0.9685
Epoch 2/5
60000/60000 [==============================] - 14s 229us/sample - loss: 0.0975 - accuracy: 0.9698 - val_loss: 0.0787 - val_accuracy: 0.9758
Epoch 3/5
60000/60000 [==============================] - 14s 231us/sample - loss: 0.0718 - accuracy: 0.9771 - val_loss: 0.0698 - val_accuracy: 0.9781
Epoch 4/5
60000/60000 [==============================] - 14s 227us/sample - loss: 0.0540 - accuracy: 0.9820 - val_loss: 0.0685 - val_accuracy: 0.9795
Epoch 5/5
60000/60000 [==============================] - 14s 228us/sample - loss: 0.0433 - accuracy: 0.9862 - val_loss: 0.0623 - val_accuracy: 0.9823


명령줄 또는 노트북 환경 내에서 TensorBoard를 시작합니다. 두 인터페이스는 대체로 동일합니다. 노트북에서는 `%tensorboard` 라인 매직을 사용합니다. 명령줄에서는 "%" 없이 동일한 명령을 실행합니다.

In [ ]:
%tensorboard --logdir logs/fit

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/quickstart_model_fit.png?raw=1"/> -->

표시된 대시보드에 대한 간략한 개요(상단 탐색 표시줄의 탭):

- **Scalars** 대시보드에는 epoch마다 손실과 메트릭이 변화하는 양상이 표시됩니다. 이를 사용하여 훈련 속도, 학습률 및 기타 스칼라 값을 추적할 수도 있습니다.
- **Graphs** 대시보드는 모델을 시각화하는 데 도움을 줍니다. 이 경우 Keras 레이어가 그래프로 표시되어 올바르게 빌드되었는지 확인할 수 있습니다.
- **Distributions** 및 **Histograms** 대시보드는 시간에 따른 텐서 분포를 보여줍니다. 이는 가중치와 바이어스를 시각화하고 예상대로 변하고 있는지 확인하는 데 유용할 수 있습니다.

다른 유형의 데이터를 로깅하면 추가 TensorBoard 플러그인이 자동으로 활성화됩니다. 예를 들어, Keras TensorBoard 콜백을 사용하면 이미지와 임베딩도 로깅할 수 있습니다. 오른쪽 상단의 "inactive" 드롭다운을 클릭하여 TensorBoard에서 사용할 수 있는 다른 플러그인을 확인할 수 있습니다.

## 기타 메서드로 TensorBoard 사용하기


[`tf.GradientTape()`](https://www.tensorflow.org/api_docs/python/tf/GradientTape)와 같은 메서드로 훈련할 때는 `tf.summary`를 사용하여 필요한 정보를 로깅합니다.

위와 동일한 데이터세트를 사용하지만, <code>tf.data.Dataset</code>로 변환하여 배치 처리 기능을 활용합니다.

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

훈련 코드는 [고급 빠른 시작](https://www.tensorflow.org/tutorials/quickstart/advanced) 가이드를 따르지만 측정 항목을 TensorBoard에 로깅하는 방법을 보여줍니다. 다음과 같이 손실 및 옵티마이저를 선택합니다.

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

훈련 중에 값을 축적하는 데 사용할 수 있고 언제든지 기록할 수 있는 상태 저장 메트릭을 만듭니다.

In [ ]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

훈련 및 테스트 함수를 정의합니다.

In [ ]:
def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

요약 작성기를 설정하여 디스크의 다른 로그 디렉토리에 요약을 작성합니다.

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

훈련을 시작합니다. `tf.summary.scalar()`를 사용하여 디스크에 요약을 작성하는 요약 작성기의 범위 내에서 훈련/테스트 중에 메트릭(손실 및 정확성)을 로깅합니다. 로깅할 메트릭과 그 빈도를 조정할 수 있습니다. 다른 유형의 데이터 로깅에는 다른 `tf.summary` 함수를 사용할 수 있습니다.

In [ ]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.24321186542510986, Accuracy: 92.84333801269531, Test Loss: 0.13006582856178284, Test Accuracy: 95.9000015258789
Epoch 2, Loss: 0.10446818172931671, Accuracy: 96.84833526611328, Test Loss: 0.08867532759904861, Test Accuracy: 97.1199951171875
Epoch 3, Loss: 0.07096975296735764, Accuracy: 97.80166625976562, Test Loss: 0.07875105738639832, Test Accuracy: 97.48999786376953
Epoch 4, Loss: 0.05380449816584587, Accuracy: 98.34166717529297, Test Loss: 0.07712937891483307, Test Accuracy: 97.56999969482422
Epoch 5, Loss: 0.041443776339292526, Accuracy: 98.71833038330078, Test Loss: 0.07514958828687668, Test Accuracy: 97.5


TensorBoard를 다시 열고, 이번에는 새 로그 디렉토리를 가리킵니다. 훈련이 진행되는 동안 모니터링하기 위해 TensorBoard를 시작할 수도 있습니다.

In [ ]:
%tensorboard --logdir logs/gradient_tape

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/quickstart_gradient_tape.png?raw=1"/> -->

여기까지입니다! 이제 Keras 콜백과 `tf.summary`를 통해 TensorBoard를 사용하는 방법을 살펴보았고, 더 많은 사용자 정의 시나리오에서 활용해 보기를 바랍니다. 

## TensorBoard.dev: ML 실험 결과 호스팅 및 공유하기

[TensorBoard.dev](https://tensorboard.dev)는 자신의 TensorBoard 로그를 업로드하고 학술 논문, 블로그 게시물, 소셜 미디어 등에서 모든 사람과 공유할 수 있는 영구 링크를 얻을 수 있는 무료 공용 서비스입니다. 이를 통해 재현성과 협력 수준을 개선할 수 있습니다.

TensorBoard.dev를 사용하려면 다음 명령을 실행하세요.


In [ ]:
!tensorboard dev upload \
  --logdir logs/fit \
  --name "(optional) My latest experiment" \
  --description "(optional) Simple comparison of several hyperparameters" \
  --one_shot

이 호출은 colab 매직을 호출하기 위한 퍼센트 접두어(`%`) 대신 셸을 호출하기 위한 느낌표 접두어(`!`)를 사용합니다. 명령줄에서 이 명령을 호출할 때는 접두어가 필요하지 않습니다.

[여기](https://tensorboard.dev/experiment/EDZb7XgKSBKo6Gznh3i8hg/#scalars)에서 예를 살펴보세요.

TensorBoard.dev 사용 방법에 대한 자세한 내용은 https://tensorboard.dev/#get-started를 참조하세요.